# Лабораторная 1
## Подраздел: Метод прогонки

* Cтудент: Ефимов А.В.
* Группа: М8О-307Б
* Вариант: 7

Загрузка библиотек, матрицы

In [6]:
import json

with open("task.json", "r") as json_file:
    task = json.load(json_file)

matrix = task["matrix"]
constraints = task["constraints"]

print(f"Matrix:\n{matrix}")
print(f"Constants: {constraints}")

Matrix:
[[15, 8, 0, 0, 0], [2, -15, 4, 0, 0], [0, 4, 11, 5, 0], [0, 0, -3, 16, -7], [0, 0, 0, 3, 8]]
Constants: [92, -84, -77, 15, -11]


Решение основывается на том, что матрица состоит из трех диагоналей, поэтому это
нужно проверять отдельно

In [2]:
def check_tridiagonal(matrix):
    matrix_size = len(matrix)
    for i in range(matrix_size - 2):
        for j in range(i + 2, matrix_size):
            if matrix[i][j] != 0 or matrix[j][i] != 0:
                return False
    return True

print(check_tridiagonal(matrix))

True


Чтобы не хранить всю матрицу эти диагонали можно извлечь

In [3]:
def get_diagonals(matrix):
    if not check_tridiagonal(matrix):
        raise ValueError("Matrix is not tridiagonal")

    matrix_size = len(matrix)
    
    upper = [0] * (matrix_size - 1)
    for i in range(matrix_size - 1):
        upper[i] = matrix[i][i + 1]

    mid = [0] * matrix_size
    for i in range(matrix_size):
        mid[i] = matrix[i][i]

    lower = [0] * (matrix_size - 1)
    for i in range(matrix_size - 1):
        lower[i] = matrix[i + 1][i]

    return (upper, mid, lower)

upper, mid, lower = get_diagonals(matrix)
print("Upper diagonal:", upper)
print("Mid diagonal:  ", mid)
print("Lower diagonal:", lower)

Upper diagonal: [8, 4, 5, -7]
Mid diagonal:   [15, -15, 11, 16, 8]
Lower diagonal: [2, 4, -3, 3]


$P_i = \frac{c_i}{b_i + a_iP_{i-1}}$, $Q_i = \frac{d_i - a_iQ_{i-1}}{b_i + a_iP_{i-1}}$, где $a, b, c$ - нижняя, средняя и верхняя диагонали соответственно, $d$ - массив констант. Эти коэффиценты понадобятся при подсчете.

In [4]:
def calc_p_coeffs(upper_diag, mid_diag, lower_diag, constraints):
    coef_count = len(constraints)
    coeffs = [0] * coef_count

    coeffs[0] = - upper_diag[0] / mid_diag[0]

    for i in range(1, coef_count - 1):
        coeffs[i] = - upper_diag[i] / (mid_diag[i] + lower_diag[i] * coeffs[i - 1])

    return coeffs

def calc_coeffs(upper_diag, mid_diag, lower_diag, constraints):
    lower_diag = [0] + lower_diag[:]

    coef_count = len(constraints)
    p_coeffs = calc_p_coeffs(upper_diag, mid_diag, lower_diag, constraints)
    q_coeffs = [0] * coef_count

    q_coeffs[0] = constraints[0] / mid_diag[0]

    for i in range(1, coef_count):
        dividend = constraints[i] - lower_diag[i] * q_coeffs[i - 1]
        divisor  = mid_diag[i] + lower_diag[i] * p_coeffs[i - 1]
        q_coeffs[i] = dividend / divisor

    return (p_coeffs, q_coeffs)

p_coeffs, q_coeffs = calc_coeffs(upper, mid, lower, constraints)
print("P coefficients:", p_coeffs)
print("Q coefficients:", q_coeffs)

P coefficients: [-0.5333333333333333, 0.24896265560165975, -0.41681079211345556, 0.4057869302801227, 0]
Q coefficients: [6.133333333333334, 5.991701244813278, -8.416810792113456, -0.5942130697198773, -1.0]


$x_i = P_ix_{i+1} + Q_i$, причем последний элемент равен последнему $Q$ коэффициенту.

In [5]:
def solve_tridiagonal(upper_diag, mid_diag, lower_diag, constraints):
    var_count = len(constraints)
    result = [0] * var_count

    p_coeffs, q_coeffs = calc_coeffs(upper_diag, mid_diag, lower_diag, constraints)

    result[var_count - 1] = q_coeffs[var_count - 1]

    for i in reversed(range(var_count - 1)):
        result[i] = p_coeffs[i] * result[i + 1] + q_coeffs[i]

    return result

solution = solve_tridiagonal(upper, mid, lower, constraints)
print("Solution:", solution)

Solution: [4.0, 4.0, -8.0, -1.0, -1.0]
